# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,6.05,61,100,3.09,US,1712373407
1,1,tarko-sale,64.9225,77.7850,-15.26,79,100,7.02,RU,1712373407
2,2,kerikeri,-35.2268,173.9474,19.36,76,64,4.21,NZ,1712373407
3,3,kodiak,57.7900,-152.4072,3.73,56,0,8.23,US,1712373408
4,4,tura,25.5198,90.2201,28.56,62,8,3.68,IN,1712373135


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:


# Configure the map plot
plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                                  color='Humidity', hover_cols=['City'],
                                  tiles='OSM', frame_width=700, frame_height=500)

# Display the map
hvplot.show(plot)

Launching server at http://localhost:60140


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_conditions_df = city_data_df[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27)& (city_data_df["Wind Speed"] < 4.5) & 
(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_conditions_df.dropna()

# Display sample data
ideal_conditions_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
151,151,margaret river,-33.9500,115.0667,26.46,32,0,2.33,AU,1712373442
213,213,abqaiq,25.9371,49.6776,23.63,26,0,2.10,SA,1712373456
255,255,harnai,30.1005,67.9374,21.97,15,0,2.18,PK,1712373464
304,304,chabahar,25.2919,60.6430,24.67,79,0,4.02,IR,1712373475
305,305,arraial do cabo,-22.9661,-42.0278,23.89,83,0,1.66,BR,1712373283
439,439,agua dulce,18.1333,-94.1333,23.17,80,0,3.74,MX,1712373505
485,485,la leonesa,-27.0379,-58.7035,23.31,88,0,3.20,AR,1712373514
549,549,marsh harbour,26.5412,-77.0636,21.74,48,0,2.70,BS,1712373529
575,575,acapulco de juarez,16.8634,-99.8901,24.90,78,0,3.60,MX,1712373385


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] =""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bethel,US,41.3712,-73.4140,61,
1,tarko-sale,RU,64.9225,77.7850,79,
2,kerikeri,NZ,-35.2268,173.9474,76,
3,kodiak,US,57.7900,-152.4072,56,
4,tura,IN,25.5198,90.2201,62,
...,...,...,...,...,...,...
588,acarau,BR,-2.8856,-40.1200,87,
589,sulecin,PL,52.4443,15.1168,90,
590,dem'yanovo,RU,60.3511,47.0842,69,
591,mizan teferi,ET,6.9833,35.5833,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
geoapify_key = "3bca82933cde424aab3d225b3be12558"
latitude=row["Lat"]
longitude=row["Lng"]

# Set parameters to search for a hotel
radius = 10000
params = params = {
   "lat": latitude,
    "lon": longitude,
    "apiKey": geoapify_key,
    "radius": 10000,
    "type": "accommodation",  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude=row["Lat"]
    longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"point:{longitude},{latitude}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: No hotel found
tarko-sale - nearest hotel: No hotel found
kerikeri - nearest hotel: No hotel found
kodiak - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
kraljevo - nearest hotel: No hotel found
katsuura - nearest hotel: No hotel found
bilibino - nearest hotel: No hotel found
lompoc - nearest hotel: No hotel found
onsong - nearest hotel: No hotel found
edinburgh of the seven seas - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
bich'vinta - nearest hotel: No hotel found
tiksi - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
selfoss - nearest hotel: No hotel found
port elizabeth - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
novyy urgal - nearest hotel: No hotel found
beloyarskiy - nearest hotel: No hotel found
tres arr

,City,Country,Lat,Lng,Humidity,Hotel Name
0,bethel,US,41.3712,-73.4140,61,No hotel found
1,tarko-sale,RU,64.9225,77.7850,79,No hotel found
2,kerikeri,NZ,-35.2268,173.9474,76,No hotel found
3,kodiak,US,57.7900,-152.4072,56,No hotel found
4,tura,IN,25.5198,90.2201,62,No hotel found
...,...,...,...,...,...,...
588,acarau,BR,-2.8856,-40.1200,87,No hotel found
589,sulecin,PL,52.4443,15.1168,90,No hotel found
590,dem'yanovo,RU,60.3511,47.0842,69,No hotel found
591,mizan teferi,ET,6.9833,35.5833,94,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [11]:
%%capture --no-display

# Configure the map plot
plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                              hover_cols=['Hotel Name', 'Country'], 
                              color='City', 
                              tiles='OSM', 
                              frame_width=700, 
                              frame_height=500)

# Display the map
hvplot.show(plot)